<a href="https://colab.research.google.com/github/SohyeonKim-dev/iOS_NLP/blob/main/skt_kogpt2_coreML_NLP_convert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
pip install accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
pip install coremltools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
import torch
from transformers import GPT2LMHeadModel

In [14]:
import coremltools as ct
'for iOS coreML'

'for iOS coreML'

In [15]:
import numpy as np

In [16]:
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
bos_token='</s>', eos_token='</s>', unk_token='<unk>',
pad_token='<pad>', mask_token='<mask>')
tokenizer.tokenize("안녕하세요. 한국어 GPT-2 입니다.😅:)l^o")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


['▁안녕',
 '하',
 '세',
 '요.',
 '▁한국어',
 '▁G',
 'P',
 'T',
 '-2',
 '▁입',
 '니다.',
 '😅',
 ':)',
 'l^o']

In [21]:
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')
text = '딸기 케이크 레시피'
input_ids = tokenizer.encode(text, return_tensors='pt')
gen_ids = model.generate(input_ids,
                           max_length=128,
                           repetition_penalty=2.0,
                           pad_token_id=tokenizer.pad_token_id,
                           eos_token_id=tokenizer.eos_token_id,
                           bos_token_id=tokenizer.bos_token_id,
                           use_cache=True)

generated = tokenizer.decode(gen_ids[0])
print(generated)

딸기 케이크 레시피를 공개했다.
이번 제품은 ‘레몬케익’, ‘오렌지치즈’ 등 3가지 맛으로 구성됐다.
‘오렌지·블루베리’는 블루베리와 함께 먹으면 더욱 달콤한 맛을 느낄 수 있다.
또한 오리지널과 오렌지 2가지 맛이 어우러진 제품으로, 취향에 따라 선택할 수도 있다. 가격은 각각 1만5000원, 2만8500원이다.
롯데제과는 지난해 11월부터 껌, 캔디, 초콜릿 등의 과자류 제품을 선보이고 있으며, 올해는 과일맛의 신제품인 '초코칩'을 출시할 계획이다.
지난달에는 초콜렛을 활용한 음료도 선보였다.
'바나


In [37]:
token_predictor = model.eval()

In [38]:
random_tokens = torch.randint(100, (5,))
traced_token_predictor = torch.jit.trace(token_predictor, random_tokens)

In [42]:
class FinishMySentence(torch.nn.Module):
    def __init__(self, model=None, eos=198):
        super(FinishMySentence, self).__init__()
        self.eos = torch.tensor([eos])
        self.next_token_predictor = model
        self.default_token = torch.tensor([0])
    
    def forward(self, x):
        sentence = x
        token = self.default_token
        while token != self.eos:
            predictions, _ = self.next_token_predictor(sentence)
            token = torch.argmax(predictions[-1, :], dim=0, keepdim=True)
            sentence = torch.cat((sentence, token), 0)
        
        return sentence

In [43]:
model = FinishMySentence(model = traced_token_predictor)
scripted_model = torch.jit.script(model)

In [44]:
mlmodel = ct.convert(
    scripted_model,
    inputs=[ct.TensorType(name="context", shape=(ct.RangeDim(1, 128),), dtype=np.int32)],
)

Converting Frontend ==> MIL Ops:   3%|▎         | 47/1401 [00:00<00:02, 463.12 ops/s]WARNING:root:Saving value type of float64 into a builtin type of fp32, might lose precision!

Converting Frontend ==> MIL Ops:  13%|█▎        | 184/1401 [00:00<00:02, 603.59 ops/s]WARNING:root:Saving value type of float64 into a builtin type of fp32, might lose precision!

Converting Frontend ==> MIL Ops:  17%|█▋        | 245/1401 [00:00<00:01, 591.07 ops/s]WARNING:root:Saving value type of float64 into a builtin type of fp32, might lose precision!

Converting Frontend ==> MIL Ops:  26%|██▌       | 361/1401 [00:00<00:01, 532.01 ops/s]WARNING:root:Saving value type of float64 into a builtin type of fp32, might lose precision!

Converting Frontend ==> MIL Ops:  36%|███▌      | 501/1401 [00:01<00:03, 272.42 ops/s]WARNING:root:Saving value type of float64 into a builtin type of fp32, might lose precision!

Converting Frontend ==> MIL Ops:  43%|████▎     | 608/1401 [00:01<00:02, 298.51 ops/s]WARNING:root:Sa

In [47]:
mlmodel.save("coreML_NLP_converted_mlmodel.mlmodel")

In [49]:
mlmodel.save("coreML_NLP_converted_mlmodel.mlpackage")

In [ ]:
# 출처(공식문서): https://coremltools.readme.io/docs/convert-nlp-model#convert-the-model-to-core-ml